In [ ]:
import sys
import zipfile
import os

import numpy as np
import pandas as pd
from IPython.display import display, HTML

sys.path.append(".")

from datasetutil import gerar_estatiscas_df, substituir_valores_nulos , verify_gg4cc , save_zip

In [ ]:

CSS = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(CSS))

# Metodos

In [ ]:
def read(zinput,nome_arquivo,
         file_format=".csv",
         separador=',',codificacao="iso-8859-1",quote='True',skip_coments=False):
    '''
        Ler zip com os datasets
    '''
    zf = zipfile.ZipFile(zinput) 

    for file in zipfile.ZipFile.namelist(zf):
        if file == nome_arquivo+file_format:
            if quote == 'True':
                df = pd.read_csv(zf.open(file), header=0, encoding=codificacao,
                                 sep=separador,error_bad_lines=False)#,
            else:
                df = pd.read_csv(zf.open(file), header=0, encoding=codificacao,
                                 sep=separador,error_bad_lines=False,
                                 quotechar = quote)
            if skip_coments:
                df = pd.read_csv(zf.open(file), header=0, encoding=codificacao,
                                 sep=separador,error_bad_lines=False,comment='#')
                
#             chunksize=640000000,
#             nrows=15)
            return df
# df.iloc[:, [68,2,10,11,9,28,29,13,14,15,1,16]]

In [ ]:
def procurar_duplicados(_df,a='id'):
    counts = _df[a].value_counts()
    return _df[_df[a].isin(counts.index[counts > 1])]

In [ ]:
def retornar_ds_limpo(_df,chave='id'):
    saida = []
    
    for k in _df[chave].unique():
        if len(saida) == 0:
            saida = _df[_df[chave] == k].head(1)
        else:
            saida = pd.concat([saida,_df[_df[chave] == k].head(1)])
    return saida

# Magellan

## restaurantes

### restaurantes4

In [ ]:
base_dir = 'D:'+os.sep+'Dados'+os.sep+'OneDrive'+os.sep+'Doutorado'+os.sep
ds_files = base_dir + 'Datasets' + os.sep + 'Magellan Data Repository' + os.sep +'Magellan' + os.sep
out_file = base_dir + 'workspace'+os.sep+'tl_pprl2'+os.sep+'datasets'+os.sep+'restaurants'+os.sep

inputZip = 'yelp_yellowpages.zip'
data = 'labeled_data'


zip_file = 'processed_yelp-yellowpages.zip'

df = read(ds_files+inputZip,data,skip_coments=True)

In [ ]:
# gabarito
gs = df[df.gold==1].iloc[:,[1,2]]


colunas =  [ 'id','address','city','name','phone','state','zipcode' ]
n_atibutos = len(colunas) -1

inicio_atributos_ds1 = 3
fim_atributos_ds1 = inicio_atributos_ds1 + n_atibutos


atts = [1] + list(range(inicio_atributos_ds1,fim_atributos_ds1))
ds1 = df.iloc[:,atts]

atts = [2] + list(range(fim_atributos_ds1,fim_atributos_ds1+n_atibutos))
ds2 = df.iloc[:,atts]

ds1.columns = colunas
ds2.columns = colunas

In [ ]:
#nan em coluna de interiro
ds2.zipcode = ds2.zipcode.fillna(-1)
ds2.zipcode = ds2.zipcode.astype(int)
ds2.zipcode = ds2.zipcode.astype(str)
ds2.zipcode = ds2.zipcode.replace('-1', np.nan)

ds1.zipcode = ds1.zipcode.fillna(-1)
ds1.zipcode = ds1.zipcode.astype(int)
ds1.zipcode = ds1.zipcode.astype(str)
ds1.zipcode = ds1.zipcode.replace('-1', np.nan)


In [ ]:
ds1 = retornar_ds_limpo(ds1)
ds2 = retornar_ds_limpo(ds2)

#### Estatisticas

In [ ]:
nan_list1 = ds1.isnull().sum()
nan_list2 = ds2.isnull().sum()

estatiscas1 = ds1.describe(include='object')
estatiscas2 = ds2.describe(include='object')

sdf1 = gerar_estatiscas_df(nan_list1,estatiscas1)
sdf1['ds'] = ds1_name

sdf2 = gerar_estatiscas_df(nan_list2,estatiscas2)
sdf2['ds'] = ds2_name

estatiscas = pd.concat([sdf1,sdf2])#.to_csv(ds_files+'stats_abt-buy.csv',sep=';')

display(sdf1)
display(sdf2)

#### Limpando os dados

Sem valores nulos

In [ ]:
gs.columns = ['id1','id2']
gs.id1 = gs.id1.astype(str)
gs.id2 = gs.id2.astype(str)
da, db = verify_gg4cc(gs)
display(da.head(5))
display(db.head(5))

In [ ]:
remover_da_lista2 = []
remover_da_lista1 = []

for i in da.id1.unique():
    remover_da_lista2.append(da[da.id1 == i].id2.values[-1])

for i in db.id2.unique():
    remover_da_lista1.append(db[db.id2 == i].id1.values[-1])

print(remover_da_lista1)
print(remover_da_lista2)

In [ ]:
cds1 = ds1[~ds1.id.isin(remover_da_lista1)]
cds2 = ds2[~ds2.id.isin(remover_da_lista2)]

gs = gs[~gs.id1.isin(remover_da_lista1)]
gs = gs[~gs.id2.isin(remover_da_lista2)]

ds1 = cds1
ds2 = cds2

In [ ]:
da, db = verify_gg4cc(gs)
display(da.head(5))
display(db.head(5))

Dataset clean-clean

In [ ]:
save_zip(ds1,ds2,gs,estatiscas,out_file + zip_file)

### restaurantes1

In [ ]:
base_dir = 'D:'+os.sep+'Dados'+os.sep+'OneDrive'+os.sep+'Doutorado'+os.sep
ds_files = base_dir + 'Datasets' + os.sep + 'Magellan Data Repository' + os.sep +'Magellan' + os.sep
out_file = base_dir + 'workspace'+os.sep+'tl_pprl2'+os.sep+'datasets'+os.sep+'restaurants'+os.sep

inputZip = 'yelp_zomato_big.zip'
data = 'labeled_data'


zip_file = 'processed_yelp-zomato.zip'

df = read(ds_files+inputZip,data,skip_coments=True)

In [ ]:
# gabarito
gs = df[df.gold==1].iloc[:,[1,2]]

colunas =  [ 'id','NAME','PHONENUMBER','ADDRESS' ]
n_atibutos = len(colunas) -1

inicio_atributos_ds1 = 3
fim_atributos_ds1 = inicio_atributos_ds1 + n_atibutos


atts = [1] + list(range(inicio_atributos_ds1,fim_atributos_ds1))
ds1 = df.iloc[:,atts]

atts = [2] + list(range(fim_atributos_ds1,fim_atributos_ds1+n_atibutos))
ds2 = df.iloc[:,atts]

ds1.columns = colunas
ds2.columns = colunas

In [ ]:
ds1 = retornar_ds_limpo(ds1)
ds2 = retornar_ds_limpo(ds2)

#### Estatisticas

In [ ]:
nan_list1 = ds1.isnull().sum()
nan_list2 = ds2.isnull().sum()

estatiscas1 = ds1.describe(include='object')
estatiscas2 = ds2.describe(include='object')

sdf1 = gerar_estatiscas_df(nan_list1,estatiscas1)
sdf1['ds'] = ds1_name

sdf2 = gerar_estatiscas_df(nan_list2,estatiscas2)
sdf2['ds'] = ds2_name

estatiscas = pd.concat([sdf1,sdf2])#.to_csv(ds_files+'stats_abt-buy.csv',sep=';')

display(sdf1)
display(sdf2)

#### Limpando os dados

Sem valores nulos

In [ ]:
gs.columns = ['id1','id2']
gs.id1 = gs.id1.astype(str)
gs.id2 = gs.id2.astype(str)
da, db = verify_gg4cc(gs)
display(da.head(5))
display(db.head(5))

In [ ]:
remover_da_lista2 = []
remover_da_lista1 = []

for i in da.id1.unique():
    remover_da_lista2.append(da[da.id1 == i].id2.values[-1])

for i in db.id2.unique():
    remover_da_lista1.append(db[db.id2 == i].id1.values[-1])

print(remover_da_lista1)
print(remover_da_lista2)

In [ ]:
cds1 = ds1[~ds1.id.isin(remover_da_lista1)]
cds2 = ds2[~ds2.id.isin(remover_da_lista2)]

gs = gs[~gs.id1.isin(remover_da_lista1)]
gs = gs[~gs.id2.isin(remover_da_lista2)]

ds1 = cds1
ds2 = cds2

In [ ]:
da, db = verify_gg4cc(gs)
display(da.head(5))
display(db.head(5))

Dataset clean-clean

In [ ]:
save_zip(ds1,ds2,gs,estatiscas,out_file + zip_file)

## Books

### Books1

In [ ]:
base_dir = 'D:'+os.sep+'Dados'+os.sep+'OneDrive'+os.sep+'Doutorado'+os.sep
ds_files = base_dir + 'Datasets' + os.sep + 'Magellan Data Repository' + os.sep +'Magellan' + os.sep
out_file = base_dir + 'workspace'+os.sep+'tl_pprl2'+os.sep+'datasets'+os.sep+'books'+os.sep

inputZip = 'books1.zip'
data = 'labeled_data'


zip_file = 'processed_amazon-barnesnobel-small.zip'

df = read(ds_files+inputZip,data,skip_coments=True)

In [ ]:
# gabarito
gs = df[df.gold_label==1].iloc[:,[1,2]]

colunas =  [ 'id','title','pubyear','pages' ]
n_atibutos = len(colunas) -1

inicio_atributos_ds1 = 3
fim_atributos_ds1 = inicio_atributos_ds1 + n_atibutos


atts = [1] + list(range(inicio_atributos_ds1,fim_atributos_ds1))
ds1 = df.iloc[:,atts]

atts = [2] + list(range(fim_atributos_ds1,fim_atributos_ds1+n_atibutos))
ds2 = df.iloc[:,atts]

ds1.columns = colunas
ds2.columns = colunas

In [ ]:
ds1.pages = ds1.pages.fillna(-1)
ds1.pages = ds1.pages.astype(int)
ds1.pages = ds1.pages.astype(str)
ds1.pages = ds1.pages.replace('-1', np.nan)

ds2.pages = ds2.pages.fillna(-1)
ds2.pages = ds2.pages.astype(int)
ds2.pages = ds2.pages.astype(str)
ds2.pages = ds2.pages.replace('-1', np.nan)


In [ ]:
ds1 = retornar_ds_limpo(ds1)
ds2 = retornar_ds_limpo(ds2)

#### Estatisticas

In [ ]:
nan_list1 = ds1.isnull().sum()
nan_list2 = ds2.isnull().sum()

estatiscas1 = ds1.describe(include='object')
estatiscas2 = ds2.describe(include='object')

sdf1 = gerar_estatiscas_df(nan_list1,estatiscas1)
sdf1['ds'] = ds1_name

sdf2 = gerar_estatiscas_df(nan_list2,estatiscas2)
sdf2['ds'] = ds2_name

estatiscas = pd.concat([sdf1,sdf2])#.to_csv(ds_files+'stats_abt-buy.csv',sep=';')

display(sdf1)
display(sdf2)

#### Limpando os dados

Sem valores nulos

In [ ]:
gs.columns = ['id1','id2']
gs.id1 = gs.id1.astype(str)
gs.id2 = gs.id2.astype(str)
da, db = verify_gg4cc(gs)
display(da.head(5))
display(db.head(5))

Dataset clean-clean

In [ ]:
save_zip(ds1,ds2,gs,estatiscas,out_file + zip_file)

### Books2

In [ ]:
base_dir = 'D:'+os.sep+'Dados'+os.sep+'OneDrive'+os.sep+'Doutorado'+os.sep
ds_files = base_dir + 'Datasets' + os.sep + 'Magellan Data Repository' + os.sep +'Magellan' + os.sep
out_file = base_dir + 'workspace'+os.sep+'tl_pprl2'+os.sep+'datasets'+os.sep+'books'+os.sep

inputZip = 'books2.zip'
data = 'labeled_data'


zip_file = 'processed_goodreads-barnesnobel.zip'

df = read(ds_files+inputZip,data,skip_coments=True)

In [ ]:
df.columns

In [ ]:
# gabarito
gs = df[df.match_label==1].iloc[:,[1,2]]

colunas_1 =  [ 'id', 'FirstAuthor', 'ISBN',
       'ISBN13', 'PageCount', 'PublishDate',
       'Publisher', 'SecondAuthor', 'ThirdAuthor',
       'Title' ]

colunas_2 =  [ 'id', 'FirstAuthor', 'SecondAuthor',
       'ThirdAuthor', 'ISBN13', 'PageCount',
       'PublishDate', 'Publisher', 'Title' ]

n_atibutos = len(colunas_1) -1

inicio_atributos_ds1 = 3
fim_atributos_ds1 = inicio_atributos_ds1 + n_atibutos


atts = [1] + list(range(inicio_atributos_ds1,fim_atributos_ds1))
ds1 = df.iloc[:,atts]

n_atibutos = len(colunas_2) -1
atts = [2] + list(range(fim_atributos_ds1,fim_atributos_ds1+n_atibutos))
ds2 = df.iloc[:,atts]

ds1.columns = colunas_1
ds2.columns = colunas_2

In [ ]:
ds1 = retornar_ds_limpo(ds1)
ds2 = retornar_ds_limpo(ds2)

#### Estatisticas

In [ ]:
nan_list1 = ds1.isnull().sum()
nan_list2 = ds2.isnull().sum()

estatiscas1 = ds1.describe(include='object')
estatiscas2 = ds2.describe(include='object')

sdf1 = gerar_estatiscas_df(nan_list1,estatiscas1)
sdf1['ds'] = ds1_name

sdf2 = gerar_estatiscas_df(nan_list2,estatiscas2)
sdf2['ds'] = ds2_name

estatiscas = pd.concat([sdf1,sdf2])#.to_csv(ds_files+'stats_abt-buy.csv',sep=';')

display(sdf1)
display(sdf2)

#### Limpando os dados

Sem valores nulos

In [ ]:
gs.columns = ['id1','id2']
gs.id1 = gs.id1.astype(str)
gs.id2 = gs.id2.astype(str)
da, db = verify_gg4cc(gs)
display(da.head(5))
display(db.head(5))

In [ ]:
remover_da_lista2 = []
remover_da_lista1 = []

for i in da.id1.unique():
    remover_da_lista2.append(da[da.id1 == i].id2.values[-1])

for i in db.id2.unique():
    remover_da_lista1.append(db[db.id2 == i].id1.values[-1])

print(remover_da_lista1)
print(remover_da_lista2)

In [ ]:
cds1 = ds1[~ds1.id.isin(remover_da_lista1)]
cds2 = ds2[~ds2.id.isin(remover_da_lista2)]

gs = gs[~gs.id1.isin(remover_da_lista1)]
gs = gs[~gs.id2.isin(remover_da_lista2)]

ds1 = cds1
ds2 = cds2

In [ ]:
da, db = verify_gg4cc(gs)
display(da.head(5))
display(db.head(5))

Dataset clean-clean

In [ ]:
save_zip(ds1,ds2,gs,estatiscas,out_file + zip_file)

### Books4

In [ ]:
base_dir = 'D:'+os.sep+'Dados'+os.sep+'OneDrive'+os.sep+'Doutorado'+os.sep
ds_files = base_dir + 'Datasets' + os.sep + 'Magellan Data Repository' + os.sep +'Magellan' + os.sep
out_file = base_dir + 'workspace'+os.sep+'tl_pprl2'+os.sep+'datasets'+os.sep+'restaurants'+os.sep

inputZip = 'books4.zip'
data = 'labeled_data'


zip_file = 'processed_amazon-barnesnobel.zip'

df = read(ds_files+inputZip,data,skip_coments=True)

In [ ]:
df.columns

In [ ]:
# gabarito
gs = df[df.gold==1].iloc[:,[1,2]]

colunas_1 =  [ 'id', 'Title', 'Author', 'ISBN_13', 'Publisher' ]

colunas_2 =  colunas_1

n_atibutos = len(colunas_1) -1

inicio_atributos_ds1 = 3
fim_atributos_ds1 = inicio_atributos_ds1 + n_atibutos


atts = [1] + list(range(inicio_atributos_ds1,fim_atributos_ds1))
ds1 = df.iloc[:,atts]

n_atibutos = len(colunas_2) -1
atts = [2] + list(range(fim_atributos_ds1,fim_atributos_ds1+n_atibutos))
ds2 = df.iloc[:,atts]

ds1.columns = colunas_1
ds2.columns = colunas_2

In [ ]:
ds1 = retornar_ds_limpo(ds1)
ds2 = retornar_ds_limpo(ds2)

#### Estatisticas

In [ ]:
nan_list1 = ds1.isnull().sum()
nan_list2 = ds2.isnull().sum()

estatiscas1 = ds1.describe(include='object')
estatiscas2 = ds2.describe(include='object')

sdf1 = gerar_estatiscas_df(nan_list1,estatiscas1)
sdf1['ds'] = ds1_name

sdf2 = gerar_estatiscas_df(nan_list2,estatiscas2)
sdf2['ds'] = ds2_name

estatiscas = pd.concat([sdf1,sdf2])#.to_csv(ds_files+'stats_abt-buy.csv',sep=';')

display(sdf1)
display(sdf2)

#### Limpando os dados

Sem valores nulos

In [ ]:
gs.columns = ['id1','id2']
gs.id1 = gs.id1.astype(str)
gs.id2 = gs.id2.astype(str)
da, db = verify_gg4cc(gs)
display(da.head(5))
display(db.head(5))

In [ ]:
remover_da_lista2 = []
remover_da_lista1 = []

for i in da.id1.unique():
    remover_da_lista2.append(da[da.id1 == i].id2.values[-1])

for i in db.id2.unique():
    remover_da_lista1.append(db[db.id2 == i].id1.values[-1])

print(remover_da_lista1)
print(remover_da_lista2)

In [ ]:
cds1 = ds1[~ds1.id.isin(remover_da_lista1)]
cds2 = ds2[~ds2.id.isin(remover_da_lista2)]

gs = gs[~gs.id1.isin(remover_da_lista1)]
gs = gs[~gs.id2.isin(remover_da_lista2)]

ds1 = cds1
ds2 = cds2

In [ ]:
da, db = verify_gg4cc(gs)
display(da.head(5))
display(db.head(5))

Dataset clean-clean

In [ ]:
save_zip(ds1,ds2,gs,estatiscas,out_file + zip_file)

## Movies

### Movies1

In [ ]:
base_dir = 'D:'+os.sep+'Dados'+os.sep+'OneDrive'+os.sep+'Doutorado'+os.sep
ds_files = base_dir + 'Datasets' + os.sep + 'Magellan Data Repository' + os.sep +'Magellan' + os.sep
out_file = base_dir + 'workspace'+os.sep+'tl_pprl2'+os.sep+'datasets'+os.sep+'movies'+os.sep

inputZip = 'movies1.zip'
data = 'labeled_data'


zip_file = 'processed_imdb-rottentomatos.zip'

df = read(ds_files+inputZip,data,skip_coments=True)

In [ ]:
df.columns

In [ ]:
# gabarito
gs = df[df.gold==1].iloc[:,[1,2]]

colunas_1 =  [ 'id', 'Director', 'Name', 'Year' ]

colunas_2 =  colunas_1

n_atibutos = len(colunas_1) -1

inicio_atributos_ds1 = 3
fim_atributos_ds1 = inicio_atributos_ds1 + n_atibutos


atts = [1] + list(range(inicio_atributos_ds1,fim_atributos_ds1))
ds1 = df.iloc[:,atts]

n_atibutos = len(colunas_2) -1
atts = [2] + list(range(fim_atributos_ds1,fim_atributos_ds1+n_atibutos))
ds2 = df.iloc[:,atts]

ds1.columns = colunas_1
ds2.columns = colunas_2

In [ ]:
ds1.Year = ds1.Year.astype(str)
ds2.Year = ds2.Year.astype(str)

In [ ]:
ds1 = retornar_ds_limpo(ds1)
ds2 = retornar_ds_limpo(ds2)

#### Estatisticas

In [ ]:
nan_list1 = ds1.isnull().sum()
nan_list2 = ds2.isnull().sum()

estatiscas1 = ds1.describe(include='object')
estatiscas2 = ds2.describe(include='object')

sdf1 = gerar_estatiscas_df(nan_list1,estatiscas1)
sdf1['ds'] = ds1_name

sdf2 = gerar_estatiscas_df(nan_list2,estatiscas2)
sdf2['ds'] = ds2_name

estatiscas = pd.concat([sdf1,sdf2])#.to_csv(ds_files+'stats_abt-buy.csv',sep=';')

display(sdf1)
display(sdf2)

#### Limpando os dados

Sem valores nulos

In [ ]:
gs.columns = ['id1','id2']
gs.id1 = gs.id1.astype(str)
gs.id2 = gs.id2.astype(str)
da, db = verify_gg4cc(gs)
display(da.head(5))
display(db.head(5))

In [ ]:
remover_da_lista2 = []
remover_da_lista1 = []

for i in da.id1.unique():
    remover_da_lista2.append(da[da.id1 == i].id2.values[-1])

for i in db.id2.unique():
    remover_da_lista1.append(db[db.id2 == i].id1.values[-1])

print(remover_da_lista1)
print(remover_da_lista2)

In [ ]:
cds1 = ds1[~ds1.id.isin(remover_da_lista1)]
cds2 = ds2[~ds2.id.isin(remover_da_lista2)]

gs = gs[~gs.id1.isin(remover_da_lista1)]
gs = gs[~gs.id2.isin(remover_da_lista2)]

ds1 = cds1
ds2 = cds2

In [ ]:
da, db = verify_gg4cc(gs)
display(da.head(5))
display(db.head(5))

Dataset clean-clean

In [ ]:
save_zip(ds1,ds2,gs,estatiscas,out_file + zip_file)

### Movies2

In [ ]:
base_dir = 'D:'+os.sep+'Dados'+os.sep+'OneDrive'+os.sep+'Doutorado'+os.sep
ds_files = base_dir + 'Datasets' + os.sep + 'Magellan Data Repository' + os.sep +'Magellan' + os.sep
out_file = base_dir + 'workspace'+os.sep+'tl_pprl2'+os.sep+'datasets'+os.sep+'movies'+os.sep

inputZip = 'movies2.zip'
data = 'labeled_data'


zip_file = 'processed_imdb-tmd.zip'

df = read(ds_files+inputZip,data,skip_coments=True)

In [ ]:
df.columns

In [ ]:
# gabarito
gs = df[df.class_label==1].iloc[:,[1,2]]

colunas_1 =  [ 'id', 'title', 'year' ]

colunas_2 =  colunas_1

n_atibutos = len(colunas_1) -1

inicio_atributos_ds1 = 3
fim_atributos_ds1 = inicio_atributos_ds1 + n_atibutos


atts = [1] + list(range(inicio_atributos_ds1,fim_atributos_ds1))
ds1 = df.iloc[:,atts]

n_atibutos = len(colunas_2) -1
atts = [2] + list(range(fim_atributos_ds1,fim_atributos_ds1+n_atibutos))
ds2 = df.iloc[:,atts]

ds1.columns = colunas_1
ds2.columns = colunas_2

In [ ]:
ds2.Year = ds2.Year.astype(str)
ds2.Year = ds2.Year.astype(str)

In [ ]:
ds1 = retornar_ds_limpo(ds1)
ds2 = retornar_ds_limpo(ds2)

#### Estatisticas

In [ ]:
nan_list1 = ds1.isnull().sum()
nan_list2 = ds2.isnull().sum()

estatiscas1 = ds1.describe(include='object')
estatiscas2 = ds2.describe(include='object')

sdf1 = gerar_estatiscas_df(nan_list1,estatiscas1)
sdf1['ds'] = ds1_name

sdf2 = gerar_estatiscas_df(nan_list2,estatiscas2)
sdf2['ds'] = ds2_name

estatiscas = pd.concat([sdf1,sdf2])#.to_csv(ds_files+'stats_abt-buy.csv',sep=';')

display(sdf1)
display(sdf2)

#### Limpando os dados

Sem valores nulos

In [ ]:
gs.columns = ['id1','id2']
gs.id1 = gs.id1.astype(str)
gs.id2 = gs.id2.astype(str)
da, db = verify_gg4cc(gs)
display(da.head(5))
display(db.head(5))

In [ ]:
remover_da_lista2 = []
remover_da_lista1 = []

for i in da.id1.unique():
    remover_da_lista2.append(da[da.id1 == i].id2.values[-1])

for i in db.id2.unique():
    remover_da_lista1.append(db[db.id2 == i].id1.values[-1])

print(remover_da_lista1)
print(remover_da_lista2)

In [ ]:
cds1 = ds1[~ds1.id.isin(remover_da_lista1)]
cds2 = ds2[~ds2.id.isin(remover_da_lista2)]

gs = gs[~gs.id1.isin(remover_da_lista1)]
gs = gs[~gs.id2.isin(remover_da_lista2)]

ds1 = cds1
ds2 = cds2

In [ ]:
da, db = verify_gg4cc(gs)
display(da.head(5))
display(db.head(5))

Dataset clean-clean

In [ ]:
save_zip(ds1,ds2,gs,estatiscas,out_file + zip_file)

### Movies2

In [ ]:
base_dir = 'D:'+os.sep+'Dados'+os.sep+'OneDrive'+os.sep+'Doutorado'+os.sep
ds_files = base_dir + 'Datasets' + os.sep + 'Magellan Data Repository' + os.sep +'Magellan' + os.sep
out_file = base_dir + 'workspace'+os.sep+'tl_pprl2'+os.sep+'datasets'+os.sep+'movies'+os.sep

inputZip = 'movies2.zip'
data = 'labeled_data'


zip_file = 'processed_imdb-tmd.zip'

df = read(ds_files+inputZip,data,skip_coments=True)

In [ ]:
df.columns

In [ ]:
# gabarito
gs = df[df.class_label==1].iloc[:,[1,2]]

colunas_1 =  [ 'id', 'title', 'year' ]

colunas_2 =  colunas_1

n_atibutos = len(colunas_1) -1

inicio_atributos_ds1 = 3
fim_atributos_ds1 = inicio_atributos_ds1 + n_atibutos


atts = [1] + list(range(inicio_atributos_ds1,fim_atributos_ds1))
ds1 = df.iloc[:,atts]

n_atibutos = len(colunas_2) -1
atts = [2] + list(range(fim_atributos_ds1,fim_atributos_ds1+n_atibutos))
ds2 = df.iloc[:,atts]

ds1.columns = colunas_1
ds2.columns = colunas_2

In [ ]:
ds2.Year = ds2.Year.astype(str)
ds2.Year = ds2.Year.astype(str)

In [ ]:
ds1 = retornar_ds_limpo(ds1)
ds2 = retornar_ds_limpo(ds2)

#### Estatisticas

In [ ]:
nan_list1 = ds1.isnull().sum()
nan_list2 = ds2.isnull().sum()

estatiscas1 = ds1.describe(include='object')
estatiscas2 = ds2.describe(include='object')

sdf1 = gerar_estatiscas_df(nan_list1,estatiscas1)
sdf1['ds'] = ds1_name

sdf2 = gerar_estatiscas_df(nan_list2,estatiscas2)
sdf2['ds'] = ds2_name

estatiscas = pd.concat([sdf1,sdf2])#.to_csv(ds_files+'stats_abt-buy.csv',sep=';')

display(sdf1)
display(sdf2)

#### Limpando os dados

Sem valores nulos

In [ ]:
gs.columns = ['id1','id2']
gs.id1 = gs.id1.astype(str)
gs.id2 = gs.id2.astype(str)
da, db = verify_gg4cc(gs)
display(da.head(5))
display(db.head(5))

In [ ]:
remover_da_lista2 = []
remover_da_lista1 = []

for i in da.id1.unique():
    remover_da_lista2.append(da[da.id1 == i].id2.values[-1])

for i in db.id2.unique():
    remover_da_lista1.append(db[db.id2 == i].id1.values[-1])

print(remover_da_lista1)
print(remover_da_lista2)

In [ ]:
cds1 = ds1[~ds1.id.isin(remover_da_lista1)]
cds2 = ds2[~ds2.id.isin(remover_da_lista2)]

gs = gs[~gs.id1.isin(remover_da_lista1)]
gs = gs[~gs.id2.isin(remover_da_lista2)]

ds1 = cds1
ds2 = cds2

In [ ]:
da, db = verify_gg4cc(gs)
display(da.head(5))
display(db.head(5))

Dataset clean-clean

In [ ]:
save_zip(ds1,ds2,gs,estatiscas,out_file + zip_file)

# Corleone

## restaurants fodors-zagats

In [ ]:
base_dir = 'D:'+os.sep+'Dados'+os.sep+'OneDrive'+os.sep+'Doutorado'+os.sep
ds_files = base_dir + 'Datasets' + os.sep + 'Magellan Data Repository' + os.sep +'Corleone' + os.sep
out_file = base_dir + 'workspace'+os.sep+'tl_pprl2'+os.sep+'datasets'+os.sep+'restaurants'+os.sep

inputZip = 'restaurants.zip'
ds1_name = 'fodors'
ds2_name = 'zagats'
gs_name ='matches_fodors_zagats'


zip_file = 'processed_fodors-zagats.zip'

ds1 = read(ds_files+inputZip,ds1_name,quote="'")
ds2 = read(ds_files+inputZip,ds2_name,quote="'")
gs = read(ds_files+inputZip,gs_name)

### Limpando

In [ ]:
# ds1.pop('title')
# z = ds2.pop('title')
# del z

ds1.name = ds1.name.astype(str)
ds1.addr = ds1.addr.astype(str)
ds1.city = ds1.city.astype(str)
ds1.phone = ds1.phone.astype(str)

ds2.name = ds2.name.astype(str)
ds2.addr = ds2.addr.astype(str)
ds2.city = ds2.city.astype(str)
ds2.phone = ds2.phone.astype(str)

### Estatisticas

In [ ]:
nan_list1 = ds1.isnull().sum()
nan_list2 = ds2.isnull().sum()

estatiscas1 = ds1.describe(include='object')
estatiscas2 = ds2.describe(include='object')

sdf1 = gerar_estatiscas_df(nan_list1,estatiscas1)
sdf1['ds'] = ds1_name

sdf2 = gerar_estatiscas_df(nan_list2,estatiscas2)
sdf2['ds'] = ds2_name

estatiscas = pd.concat([sdf1,sdf2])#.to_csv(ds_files+'stats_abt-buy.csv',sep=';')

display(sdf1)
display(sdf2)

### Limpando os dados

Sem valores nulos

In [ ]:
gs.columns = ['id1','id2']
gs.id1 = gs.id1.astype(str)
gs.id2 = gs.id2.astype(str)
da, db = verify_gg4cc(gs)
display(da.head(5))
display(db.head(5))

Dataset clean-clean

### Salvando os dados

In [ ]:
save_zip(ds1,ds2,gs,estatiscas,out_file + zip_file)